In [4]:
import findspark
#adding PySpark to sys.path at runtime.using package findspark 
findspark.init('C:\spark')

In [8]:
import sys
print(sys.path)

['C:\\spark\\python', 'C:\\Users\\Uttam\\AppData\\Local\\Temp\\spark-268e66d8-be55-4238-b1ba-b1519ae044e6\\userFiles-750086e2-eec3-48b2-97a0-f1a14ac7380e', 'C:\\spark\\python\\lib\\py4j-0.10.9-src.zip', 'c:\\users\\uttam\\anaconda3\\envs\\pyspark\\python36.zip', 'c:\\users\\uttam\\anaconda3\\envs\\pyspark\\DLLs', 'c:\\users\\uttam\\anaconda3\\envs\\pyspark\\lib', 'c:\\users\\uttam\\anaconda3\\envs\\pyspark', '', 'c:\\users\\uttam\\anaconda3\\envs\\pyspark\\lib\\site-packages', 'c:\\users\\uttam\\anaconda3\\envs\\pyspark\\lib\\site-packages\\win32', 'c:\\users\\uttam\\anaconda3\\envs\\pyspark\\lib\\site-packages\\win32\\lib', 'c:\\users\\uttam\\anaconda3\\envs\\pyspark\\lib\\site-packages\\Pythonwin', 'c:\\users\\uttam\\anaconda3\\envs\\pyspark\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Uttam\\.ipython']


In [5]:
import pyspark
from pyspark.sql import SparkSession
spark= SparkSession.builder.appName('Customers').getOrCreate()

In [6]:
spark

In [11]:
from pyspark.ml.regression import LinearRegression

In [13]:
data = spark.read.csv('Ecommerce_Customers.csv',inferSchema=True,header=True)
data

DataFrame[Email: string, Address: string, Avg Session Length: double, Time on App: double, Time on Website: double, Length of Membership: double, Yearly Amount Spent: double]

In [14]:
data.show(5)

+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+
|               Email|             Address|Avg Session Length|Time on App|Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|       34.49726773|12.65565115|    39.57766802|         4.082620633|         587.951054|
|   hduke@hotmail.com|4547 Archer Commo...|       31.92627203|11.10946073|    37.26895887|         2.664034182|        392.2049334|
|    pallen@yahoo.com|24645 Valerie Uni...|       33.00091476|11.33027806|    37.11059744|         4.104543202|        487.5475049|
|riverarebecca@gma...|1414 David Throug...|       34.30555663|13.71751367|    36.72128268|         3.120178783|         581.852344|
|mstephens@davidso...|14023 Rodriguez P...|       33.33067252|12.79518855|  

In [15]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



sklearn

x1,X2,X3,X4,X5    Y1 ---->model-->prediction

[X1,X2,X3,X4,X5]  Y1---->model--->prediction

In [16]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [18]:
featureassembler=VectorAssembler(inputCols=["Avg Session Length","Time on App","Time on Website","Length of Membership"],
                                 outputCol="Independent Features")
featureassembler

VectorAssembler_85da698f78f2

In [19]:
output = featureassembler.transform(data)
output.show(5)

+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+--------------------+
|               Email|             Address|Avg Session Length|Time on App|Time on Website|Length of Membership|Yearly Amount Spent|Independent Features|
+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|       34.49726773|12.65565115|    39.57766802|         4.082620633|         587.951054|[34.49726773,12.6...|
|   hduke@hotmail.com|4547 Archer Commo...|       31.92627203|11.10946073|    37.26895887|         2.664034182|        392.2049334|[31.92627203,11.1...|
|    pallen@yahoo.com|24645 Valerie Uni...|       33.00091476|11.33027806|    37.11059744|         4.104543202|        487.5475049|[33.00091476,11.3...|
|riverarebecca@gma...|1414 David Throug...|       34.30555663|13.71751367|    36.7

In [20]:
output.select('Independent Features').show(10)

+--------------------+
|Independent Features|
+--------------------+
|[34.49726773,12.6...|
|[31.92627203,11.1...|
|[33.00091476,11.3...|
|[34.30555663,13.7...|
|[33.33067252,12.7...|
|[33.87103788,12.0...|
|[32.0215955,11.36...|
|[32.73914294,12.3...|
|[33.9877729,13.38...|
|[31.93654862,11.8...|
+--------------------+
only showing top 10 rows



In [21]:
output.columns

['Email',
 'Address',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent',
 'Independent Features']

In [22]:
finalized_data=output.select("Independent Features","Yearly Amount Spent")
finalized_data.show(10)

+--------------------+-------------------+
|Independent Features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.49726773,12.6...|         587.951054|
|[31.92627203,11.1...|        392.2049334|
|[33.00091476,11.3...|        487.5475049|
|[34.30555663,13.7...|         581.852344|
|[33.33067252,12.7...|         599.406092|
|[33.87103788,12.0...|        637.1024479|
|[32.0215955,11.36...|        521.5721748|
|[32.73914294,12.3...|        549.9041461|
|[33.9877729,13.38...|         570.200409|
|[31.93654862,11.8...|        427.1993849|
+--------------------+-------------------+
only showing top 10 rows



In [26]:
finalized_data.count()

500

In [23]:
train_data,test_data=finalized_data.randomSplit([0.75,0.25])

In [25]:
train_data.count(), test_data.count()

(371, 129)

In [27]:
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Yearly Amount Spent')
regressor=regressor.fit(train_data)

In [28]:
regressor.coefficients

DenseVector([25.7614, 37.9966, 0.6181, 61.6478])

In [29]:
regressor.intercept

-1050.705498086327

In [30]:
pred_results=regressor.evaluate(test_data)

In [32]:
pred_results.predictions.show(40)

+--------------------+-------------------+------------------+
|Independent Features|Yearly Amount Spent|        prediction|
+--------------------+-------------------+------------------+
|[30.4925367,11.56...|        282.4712457|287.75268116978236|
|[30.83643267,13.1...|        467.5019004| 470.8744886214495|
|[30.87948434,13.2...|           490.2066| 493.2521129089248|
|[31.04722214,11.1...|        392.4973992|388.99242691414497|
|[31.1695068,13.97...|        427.3565308| 415.8042713383718|
|[31.26064687,13.2...|        421.3266313| 421.3272873984372|
|[31.30919264,11.9...|        432.7207178|429.79862428292085|
|[31.35847719,12.8...|        495.1759504| 490.7124990864295|
|[31.52575242,11.3...|        443.9656268| 450.1740473683358|
|[31.5261979,12.04...|        409.0945262|418.49064214368923|
|[31.65480968,13.0...|        475.2634237|468.20217208884355|
|[31.7207699,11.75...|        538.7749335| 546.5719374158718|
|[31.72420252,13.1...|        503.3878873|  508.763050263223|
|[31.809